In [43]:
import pandas as pd
from pathlib import Path
data_root = Path('.')
train_source = pd.read_parquet(data_root / 'label_train_source.parquet')
train_target = pd.read_parquet(data_root / 'label_train_target.parquet')
test_source = pd.read_parquet(data_root / 'label_test_source.parquet')
meta_song = pd.read_parquet(data_root / 'meta_song.parquet')
# meta_song_composer = pd.read_parquet(data_root / 'meta_song_composer.parquet')
# meta_song_genre = pd.read_parquet(data_root / 'meta_song_genre.parquet')
# meta_song_lyricist = pd.read_parquet(data_root / 'meta_song_lyricist.parquet')
# meta_song_producer = pd.read_parquet(data_root / 'meta_song_producer.parquet')
# meta_song_titletext = pd.read_parquet(data_root / 'meta_song_titletext.parquet')


train_source.dtypes

session_id          int64
song_id            object
unix_played_at      int64
play_status         int64
login_type          int64
listening_order     int64
dtype: object

In [44]:
# map song_id to song_index to save memory and speed up 
meta_song['song_index'] = meta_song.index
train_source = train_source.merge(meta_song[['song_id', 'song_index']], on='song_id', how='left')
train_target = train_target.merge(meta_song[['song_id', 'song_index']], on='song_id', how='left')
test_source = test_source.merge(meta_song[['song_id', 'song_index']], on='song_id', how='left')
del train_source['song_id']
del train_target['song_id']
del test_source['song_id']

In [45]:
train_source['previous1_song_index'] = train_source.groupby('session_id')['song_index'].shift(1)
train_source['previous2_song_index'] = train_source.groupby('session_id')['song_index'].shift(2)
train_source['previous3_song_index'] = train_source.groupby('session_id')['song_index'].shift(3)
train_source['previous4_song_index'] = train_source.groupby('session_id')['song_index'].shift(4)
train_source['previous5_song_index'] = train_source.groupby('session_id')['song_index'].shift(5)
test_source['previous1_song_index'] = test_source.groupby('session_id')['song_index'].shift(1)
test_source['previous2_song_index'] = test_source.groupby('session_id')['song_index'].shift(2)
test_source['previous3_song_index'] = test_source.groupby('session_id')['song_index'].shift(3)
test_source['previous4_song_index'] = test_source.groupby('session_id')['song_index'].shift(4)
test_source['previous5_song_index'] = test_source.groupby('session_id')['song_index'].shift(5)
history_fields = ['previous1_song_index', 'previous2_song_index', 'previous3_song_index', 'previous4_song_index', 'previous5_song_index', 'song_index']
history=pd.concat([train_source[history_fields], test_source[history_fields]], ignore_index=True)
history.dropna(inplace=True)
history

,previous1_song_index,previous2_song_index,previous3_song_index,previous4_song_index,previous5_song_index,song_index
5,955400.0,753834.0,753834.0,753834.0,283767.0,13609
6,13609.0,955400.0,753834.0,753834.0,753834.0,955400
7,955400.0,13609.0,955400.0,753834.0,753834.0,972005
8,972005.0,955400.0,13609.0,955400.0,753834.0,13609
9,13609.0,972005.0,955400.0,13609.0,955400.0,753834
...,...,...,...,...,...,...
14306455,154541.0,125431.0,596540.0,219518.0,618643.0,408927
14306456,408927.0,154541.0,125431.0,596540.0,219518.0,125430
14306457,125430.0,408927.0,154541.0,125431.0,596540.0,490246
14306458,490246.0,125430.0,408927.0,154541.0,125431.0,482103


In [46]:
gram1 = history[['previous1_song_index', 'song_index']].groupby(['previous1_song_index']).value_counts(normalize=True, sort=True)
gram1 = gram1.reset_index().groupby('previous1_song_index').first().reset_index()
gram1

,previous1_song_index,song_index,proportion
0,1.0,1014814,1.000000
1,3.0,87846,1.000000
2,4.0,136296,0.166667
3,5.0,5,0.086957
4,6.0,519864,1.000000
...,...,...,...
640551,1030707.0,76930,0.117647
640552,1030708.0,736280,0.192000
640553,1030709.0,915034,0.051282
640554,1030710.0,1030649,0.962025


In [47]:
gram2 = history[['previous2_song_index', 'previous1_song_index', 'song_index']].groupby(['previous2_song_index', 'previous1_song_index']).value_counts(normalize=True, sort=True)
gram2 = gram2.reset_index().groupby(['previous2_song_index','previous1_song_index']).first().reset_index()
gram2

,previous2_song_index,previous1_song_index,song_index,proportion
0,1.0,1014814.0,799780,1.0
1,3.0,87846.0,897082,1.0
2,4.0,18255.0,987697,1.0
3,4.0,136296.0,397676,1.0
4,4.0,435266.0,792055,1.0
...,...,...,...,...
7063533,1030711.0,642148.0,913721,1.0
7063534,1030711.0,643112.0,6835,1.0
7063535,1030711.0,946599.0,125355,1.0
7063536,1030711.0,967484.0,609494,1.0


In [48]:
gram3 = history[['previous3_song_index', 'previous2_song_index', 'previous1_song_index', 'song_index']].groupby(['previous3_song_index', 'previous2_song_index', 'previous1_song_index']).value_counts(normalize=True, sort=True)
gram3 = gram3.reset_index().groupby(['previous3_song_index', 'previous2_song_index','previous1_song_index']).first().reset_index()
gram3

,previous3_song_index,previous2_song_index,previous1_song_index,song_index,proportion
0,1.0,1014814.0,799780.0,938083,1.0
1,3.0,87846.0,897082.0,813298,1.0
2,3.0,906384.0,860711.0,44990,1.0
3,4.0,18255.0,987697.0,279996,1.0
4,4.0,136296.0,397676.0,47115,1.0
...,...,...,...,...,...
8478645,1030711.0,643112.0,6835.0,111132,1.0
8478646,1030711.0,936567.0,1023129.0,571476,1.0
8478647,1030711.0,946599.0,125355.0,552899,1.0
8478648,1030711.0,967484.0,609494.0,967484,1.0


In [49]:
gram4 = history[['previous4_song_index', 'previous3_song_index', 'previous2_song_index', 'previous1_song_index', 'song_index']].groupby(['previous4_song_index', 'previous3_song_index', 'previous2_song_index', 'previous1_song_index']).value_counts(normalize=True, sort=True)
gram4 = gram4.reset_index().groupby(['previous4_song_index', 'previous3_song_index', 'previous2_song_index','previous1_song_index']).first().reset_index()
gram4

,previous4_song_index,previous3_song_index,previous2_song_index,previous1_song_index,song_index,proportion
0,1.0,1014814.0,799780.0,938083.0,647966,1.0
1,3.0,87846.0,897082.0,813298.0,797523,1.0
2,3.0,401307.0,318166.0,401307.0,107851,1.0
3,3.0,906384.0,860711.0,44990.0,243158,1.0
4,4.0,18255.0,987697.0,279996.0,55469,1.0
...,...,...,...,...,...,...
8780502,1030711.0,643112.0,6835.0,111132.0,552899,1.0
8780503,1030711.0,936567.0,1023129.0,571476.0,967508,1.0
8780504,1030711.0,946599.0,125355.0,552899.0,539215,1.0
8780505,1030711.0,967484.0,609494.0,967484.0,725932,1.0


In [50]:
gram5 = history[['previous5_song_index', 'previous4_song_index', 'previous3_song_index', 'previous2_song_index', 'previous1_song_index', 'song_index']].groupby(['previous5_song_index', 'previous4_song_index', 'previous3_song_index', 'previous2_song_index', 'previous1_song_index']).value_counts(normalize=True, sort=True)
gram5 = gram5.reset_index().groupby(['previous5_song_index', 'previous4_song_index', 'previous3_song_index', 'previous2_song_index','previous1_song_index']).first().reset_index()
gram5

,previous5_song_index,previous4_song_index,previous3_song_index,previous2_song_index,previous1_song_index,song_index,proportion
0,1.0,488417.0,697989.0,416308.0,570834.0,146850,1.0
1,1.0,1014814.0,799780.0,938083.0,647966.0,589282,1.0
2,3.0,87846.0,897082.0,813298.0,797523.0,821017,1.0
3,3.0,200709.0,1021838.0,995187.0,964869.0,168421,1.0
4,3.0,401307.0,318166.0,401307.0,107851.0,318166,1.0
...,...,...,...,...,...,...,...
8950655,1030711.0,643112.0,6835.0,111132.0,552899.0,67869,1.0
8950656,1030711.0,936567.0,1023129.0,571476.0,967508.0,119291,1.0
8950657,1030711.0,946599.0,125355.0,552899.0,539215.0,561432,1.0
8950658,1030711.0,967484.0,609494.0,967484.0,725932.0,934560,1.0


In [51]:
train_X = train_source.pivot_table(index='session_id', columns='listening_order', values='song_index', aggfunc='first', fill_value=0)
train_X

listening_order,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
session_id,,,,,,,,,,,,,,,,,,,,
1,354121,1030664,642780,280721,90293,766515,769968,1007853,249587,615299,166527,973329,489807,789395,832194,962778,913263,910165,194387,722034
2,256188,144246,400761,625467,281059,880625,849960,40747,360309,123421,360309,123421,158192,534994,442771,659818,415453,307906,523383,672083
3,514974,300949,632246,353623,93125,165609,240446,380094,139121,316125,516472,389977,385637,673916,623997,332935,133500,606328,578482,261314
4,277607,219437,138361,622824,967426,700237,868800,490471,835071,628370,7085,238820,503008,238820,503008,503008,824980,267047,238820,709693
5,892347,509757,97064,66960,896666,74810,850960,422356,328649,328649,325229,672168,337713,14961,685694,7267,589816,482509,77515,246245
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715317,323105,408854,256899,285864,303707,21973,57762,514986,8113,230578,368936,514985,536976,615765,292762,506580,505242,614405,198412,422547
715320,741894,829501,899618,737589,455527,1018618,737589,899618,430854,732106,732103,1027226,967682,98899,239043,604807,758868,918865,967471,970952
715321,647388,647388,210671,918865,528842,953923,262081,102874,353437,569657,823692,42423,789301,1026608,278029,791040,791040,832786,277358,522132


In [52]:
train_Y = train_target.pivot_table(index='session_id', columns='listening_order', values='song_index', aggfunc='first', fill_value=0)
train_Y

listening_order,21,22,23,24,25
session_id,,,,,
1,624433,901147,810092,74420,250714
2,52476,249524,514925,495260,130336
3,203739,402134,129395,230525,946149
4,453065,709693,453065,709693,453065
5,186586,338120,186586,338120,130652
...,...,...,...,...,...
715317,615766,560313,553343,624094,258244
715320,784715,745441,716579,732103,732103
715321,681203,90781,692730,469116,592612


In [53]:
df = pd.concat([train_X[[16,17,18,19,20]], train_Y], axis=1)
df

listening_order,16,17,18,19,20,21,22,23,24,25
session_id,,,,,,,,,,
1,962778,913263,910165,194387,722034,624433,901147,810092,74420,250714
2,659818,415453,307906,523383,672083,52476,249524,514925,495260,130336
3,332935,133500,606328,578482,261314,203739,402134,129395,230525,946149
4,503008,824980,267047,238820,709693,453065,709693,453065,709693,453065
5,7267,589816,482509,77515,246245,186586,338120,186586,338120,130652
...,...,...,...,...,...,...,...,...,...,...
715317,506580,505242,614405,198412,422547,615766,560313,553343,624094,258244
715320,604807,758868,918865,967471,970952,784715,745441,716579,732103,732103
715321,791040,791040,832786,277358,522132,681203,90781,692730,469116,592612


In [54]:
gram1_predict = pd.merge(df, gram1, left_on=[20], right_on=['previous1_song_index'], how='left')

In [55]:
(gram1_predict['song_index'] == gram1_predict[21]).mean()

0.14782642125331363

In [56]:
gram1_predict['p'] = gram1_predict['song_index'].fillna(gram1_predict[20]).astype(int).values

In [57]:
(gram1_predict['p'] == gram1_predict[21]).mean()

0.14826678130007567

In [58]:
import rich
r = gram1_predict[[21,22,23,24,25]] == gram1_predict[['p']].values
r[25] = r[25] & ~(r[[21,22,23,24]].any(axis=1))
r[24] = r[24] & ~(r[[21,22,23]].any(axis=1))
r[23] = r[23] & ~(r[[21,22]].any(axis=1))
r[22] = r[22] & ~(r[[21]].any(axis=1))
w = [1., 0.63, 0.5, 0.43, 0.38]
all_songs = meta_song.shape[0]
dcg = r.mean() @ w
predict_songs = gram1_predict['p'].sample(frac=0.25).nunique()
fill_songs = 4*test_source['session_id'].nunique()
coverage = (predict_songs+fill_songs)/all_songs
rich.print({'dcg':dcg, 'coverage':coverage, 'total':dcg*0.8+coverage*0.2})

{'dcg': 0.16769132508182485, 'coverage': 0.599407982055123, 'total': 0.2540346564764845}

In [59]:
for k in ['song_index', 'previous1_song_index', 'proportion']:
    del gram1_predict[k]
gram1_predict

,16,17,18,19,20,21,22,23,24,25,p
0,962778,913263,910165,194387,722034,624433,901147,810092,74420,250714,722034
1,659818,415453,307906,523383,672083,52476,249524,514925,495260,130336,672083
2,332935,133500,606328,578482,261314,203739,402134,129395,230525,946149,261314
3,503008,824980,267047,238820,709693,453065,709693,453065,709693,453065,709693
4,7267,589816,482509,77515,246245,186586,338120,186586,338120,130652,365352
...,...,...,...,...,...,...,...,...,...,...,...
572254,506580,505242,614405,198412,422547,615766,560313,553343,624094,258244,198412
572255,604807,758868,918865,967471,970952,784715,745441,716579,732103,732103,745441
572256,791040,791040,832786,277358,522132,681203,90781,692730,469116,592612,340530
572257,880639,722323,825005,247342,832762,953126,766052,82260,126631,82260,716463


In [60]:
gram2_predict = pd.merge(gram1_predict, gram2, left_on=[19,20], right_on=['previous2_song_index', 'previous1_song_index'], how='left')
for k in ['previous1_song_index', 'previous2_song_index', 'proportion']:
    del gram2_predict[k]
gram2_predict

,16,17,18,19,20,21,22,23,24,25,p,song_index
0,962778,913263,910165,194387,722034,624433,901147,810092,74420,250714,722034,NaN
1,659818,415453,307906,523383,672083,52476,249524,514925,495260,130336,672083,364625.0
2,332935,133500,606328,578482,261314,203739,402134,129395,230525,946149,261314,NaN
3,503008,824980,267047,238820,709693,453065,709693,453065,709693,453065,709693,NaN
4,7267,589816,482509,77515,246245,186586,338120,186586,338120,130652,365352,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
572254,506580,505242,614405,198412,422547,615766,560313,553343,624094,258244,198412,615766.0
572255,604807,758868,918865,967471,970952,784715,745441,716579,732103,732103,745441,967471.0
572256,791040,791040,832786,277358,522132,681203,90781,692730,469116,592612,340530,NaN
572257,880639,722323,825005,247342,832762,953126,766052,82260,126631,82260,716463,953126.0


In [61]:
(gram2_predict['song_index'] == gram2_predict[21]).mean()

0.21368296523077837

In [62]:
gram2_predict['p2'] = gram2_predict['song_index'].fillna(gram2_predict['p']).astype(int)
(gram2_predict['p2'] == gram2_predict[21]).mean()

0.22413103157835876

In [63]:
cond = (gram2_predict[19] == gram2_predict[20])
gram2_predict['p3'] = gram2_predict['p2'].where(~cond, gram2_predict[20])
(gram2_predict['p3'] == gram2_predict[21]).mean()

0.22513232644659148

In [64]:
r = gram2_predict[[21,22,23,24,25]] == gram2_predict[['p2']].values
r[25] = r[25] & ~(r[[21,22,23,24]].any(axis=1))
r[24] = r[24] & ~(r[[21,22,23]].any(axis=1))
r[23] = r[23] & ~(r[[21,22]].any(axis=1))
r[22] = r[22] & ~(r[[21]].any(axis=1))
w = [1., 0.63, 0.5, 0.43, 0.38]
all_songs = meta_song.shape[0]
dcg = r.mean() @ w
predict_songs = gram2_predict['p2'].sample(frac=0.25).nunique()
fill_songs = 4*test_source['session_id'].nunique()
coverage = (predict_songs+fill_songs)/all_songs
rich.print({'dcg':dcg, 'coverage':coverage, 'total':dcg*0.8+coverage*0.2})

{'dcg': 0.24136640926573458, 'coverage': 0.6047557416620744, 'total': 0.31404427574500254}

In [99]:
gram2_predict

,16,17,18,19,20,21,22,23,24,25,p,p2
0,962778,913263,910165,194387,722034,624433,901147,810092,74420,250714,722034,722034
1,659818,415453,307906,523383,672083,52476,249524,514925,495260,130336,672083,364625
2,332935,133500,606328,578482,261314,203739,402134,129395,230525,946149,261314,261314
3,503008,824980,267047,238820,709693,453065,709693,453065,709693,453065,709693,709693
4,7267,589816,482509,77515,246245,186586,338120,186586,338120,130652,365352,365352
...,...,...,...,...,...,...,...,...,...,...,...,...
572254,506580,505242,614405,198412,422547,615766,560313,553343,624094,258244,198412,615766
572255,604807,758868,918865,967471,970952,784715,745441,716579,732103,732103,745441,967471
572256,791040,791040,832786,277358,522132,681203,90781,692730,469116,592612,340530,340530
572257,880639,722323,825005,247342,832762,953126,766052,82260,126631,82260,716463,953126


In [66]:
for k in ['song_index', 'p3']:
    del gram2_predict[k]
gram2_predict

,16,17,18,19,20,21,22,23,24,25,p,p2
0,962778,913263,910165,194387,722034,624433,901147,810092,74420,250714,722034,722034
1,659818,415453,307906,523383,672083,52476,249524,514925,495260,130336,672083,364625
2,332935,133500,606328,578482,261314,203739,402134,129395,230525,946149,261314,261314
3,503008,824980,267047,238820,709693,453065,709693,453065,709693,453065,709693,709693
4,7267,589816,482509,77515,246245,186586,338120,186586,338120,130652,365352,365352
...,...,...,...,...,...,...,...,...,...,...,...,...
572254,506580,505242,614405,198412,422547,615766,560313,553343,624094,258244,198412,615766
572255,604807,758868,918865,967471,970952,784715,745441,716579,732103,732103,745441,967471
572256,791040,791040,832786,277358,522132,681203,90781,692730,469116,592612,340530,340530
572257,880639,722323,825005,247342,832762,953126,766052,82260,126631,82260,716463,953126


In [67]:
gram3_predict = pd.merge(gram2_predict, gram3, left_on=[18,19,20], right_on=['previous3_song_index', 'previous2_song_index', 'previous1_song_index'], how='left')
gram3_predict

,16,17,18,19,20,21,22,23,24,25,p,p2,previous3_song_index,previous2_song_index,previous1_song_index,song_index,proportion
0,962778,913263,910165,194387,722034,624433,901147,810092,74420,250714,722034,722034,NaN,NaN,NaN,NaN,NaN
1,659818,415453,307906,523383,672083,52476,249524,514925,495260,130336,672083,364625,NaN,NaN,NaN,NaN,NaN
2,332935,133500,606328,578482,261314,203739,402134,129395,230525,946149,261314,261314,NaN,NaN,NaN,NaN,NaN
3,503008,824980,267047,238820,709693,453065,709693,453065,709693,453065,709693,709693,NaN,NaN,NaN,NaN,NaN
4,7267,589816,482509,77515,246245,186586,338120,186586,338120,130652,365352,365352,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572254,506580,505242,614405,198412,422547,615766,560313,553343,624094,258244,198412,615766,614405.0,198412.0,422547.0,615766.0,1.0
572255,604807,758868,918865,967471,970952,784715,745441,716579,732103,732103,745441,967471,NaN,NaN,NaN,NaN,NaN
572256,791040,791040,832786,277358,522132,681203,90781,692730,469116,592612,340530,340530,NaN,NaN,NaN,NaN,NaN
572257,880639,722323,825005,247342,832762,953126,766052,82260,126631,82260,716463,953126,825005.0,247342.0,832762.0,953126.0,1.0


In [68]:
for k in ['previous3_song_index', 'previous2_song_index', 'previous1_song_index', 'proportion']:
    del gram3_predict[k]
gram3_predict

,16,17,18,19,20,21,22,23,24,25,p,p2,song_index
0,962778,913263,910165,194387,722034,624433,901147,810092,74420,250714,722034,722034,NaN
1,659818,415453,307906,523383,672083,52476,249524,514925,495260,130336,672083,364625,NaN
2,332935,133500,606328,578482,261314,203739,402134,129395,230525,946149,261314,261314,NaN
3,503008,824980,267047,238820,709693,453065,709693,453065,709693,453065,709693,709693,NaN
4,7267,589816,482509,77515,246245,186586,338120,186586,338120,130652,365352,365352,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
572254,506580,505242,614405,198412,422547,615766,560313,553343,624094,258244,198412,615766,615766.0
572255,604807,758868,918865,967471,970952,784715,745441,716579,732103,732103,745441,967471,NaN
572256,791040,791040,832786,277358,522132,681203,90781,692730,469116,592612,340530,340530,NaN
572257,880639,722323,825005,247342,832762,953126,766052,82260,126631,82260,716463,953126,953126.0


In [70]:
(gram3_predict['song_index'] == gram3_predict[21]).mean()

0.207462005840013

In [71]:
gram3_predict['p3'] = gram3_predict['song_index'].fillna(gram3_predict['p']).astype(int)
(gram3_predict['p3'] == gram3_predict[21]).mean()

0.23559262501769304

In [72]:
gram3_predict

,16,17,18,19,20,21,22,23,24,25,p,p2,song_index,p3
0,962778,913263,910165,194387,722034,624433,901147,810092,74420,250714,722034,722034,NaN,722034
1,659818,415453,307906,523383,672083,52476,249524,514925,495260,130336,672083,364625,NaN,364625
2,332935,133500,606328,578482,261314,203739,402134,129395,230525,946149,261314,261314,NaN,261314
3,503008,824980,267047,238820,709693,453065,709693,453065,709693,453065,709693,709693,NaN,709693
4,7267,589816,482509,77515,246245,186586,338120,186586,338120,130652,365352,365352,NaN,365352
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572254,506580,505242,614405,198412,422547,615766,560313,553343,624094,258244,198412,615766,615766.0,615766
572255,604807,758868,918865,967471,970952,784715,745441,716579,732103,732103,745441,967471,NaN,967471
572256,791040,791040,832786,277358,522132,681203,90781,692730,469116,592612,340530,340530,NaN,340530
572257,880639,722323,825005,247342,832762,953126,766052,82260,126631,82260,716463,953126,953126.0,953126


In [73]:
r = gram3_predict[[21,22,23,24,25]] == gram3_predict[['p3']].values
r[25] = r[25] & ~(r[[21,22,23,24]].any(axis=1))
r[24] = r[24] & ~(r[[21,22,23]].any(axis=1))
r[23] = r[23] & ~(r[[21,22]].any(axis=1))
r[22] = r[22] & ~(r[[21]].any(axis=1))
w = [1., 0.63, 0.5, 0.43, 0.38]
all_songs = meta_song.shape[0]
dcg = r.mean() @ w
predict_songs = gram3_predict['p3'].sample(frac=0.25).nunique()
fill_songs = 4*test_source['session_id'].nunique()
coverage = (predict_songs+fill_songs)/all_songs
rich.print({'dcg':dcg, 'coverage':coverage, 'total':dcg*0.8+coverage*0.2})

{'dcg': 0.25242937201511906, 'coverage': 0.6048042518181607, 'total': 0.3229043479757274}

In [74]:
for k in ['song_index']:
    del gram3_predict[k]
gram3_predict

,16,17,18,19,20,21,22,23,24,25,p,p2,p3
0,962778,913263,910165,194387,722034,624433,901147,810092,74420,250714,722034,722034,722034
1,659818,415453,307906,523383,672083,52476,249524,514925,495260,130336,672083,364625,364625
2,332935,133500,606328,578482,261314,203739,402134,129395,230525,946149,261314,261314,261314
3,503008,824980,267047,238820,709693,453065,709693,453065,709693,453065,709693,709693,709693
4,7267,589816,482509,77515,246245,186586,338120,186586,338120,130652,365352,365352,365352
...,...,...,...,...,...,...,...,...,...,...,...,...,...
572254,506580,505242,614405,198412,422547,615766,560313,553343,624094,258244,198412,615766,615766
572255,604807,758868,918865,967471,970952,784715,745441,716579,732103,732103,745441,967471,967471
572256,791040,791040,832786,277358,522132,681203,90781,692730,469116,592612,340530,340530,340530
572257,880639,722323,825005,247342,832762,953126,766052,82260,126631,82260,716463,953126,953126


In [75]:
gram4_predict = pd.merge(gram3_predict, gram4, left_on=[17,18,19,20], right_on=['previous4_song_index', 'previous3_song_index', 'previous2_song_index', 'previous1_song_index'], how='left')
gram4_predict

,16,17,18,19,20,21,22,23,24,25,p,p2,p3,previous4_song_index,previous3_song_index,previous2_song_index,previous1_song_index,song_index,proportion
0,962778,913263,910165,194387,722034,624433,901147,810092,74420,250714,722034,722034,722034,NaN,NaN,NaN,NaN,NaN,NaN
1,659818,415453,307906,523383,672083,52476,249524,514925,495260,130336,672083,364625,364625,NaN,NaN,NaN,NaN,NaN,NaN
2,332935,133500,606328,578482,261314,203739,402134,129395,230525,946149,261314,261314,261314,NaN,NaN,NaN,NaN,NaN,NaN
3,503008,824980,267047,238820,709693,453065,709693,453065,709693,453065,709693,709693,709693,NaN,NaN,NaN,NaN,NaN,NaN
4,7267,589816,482509,77515,246245,186586,338120,186586,338120,130652,365352,365352,365352,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572254,506580,505242,614405,198412,422547,615766,560313,553343,624094,258244,198412,615766,615766,NaN,NaN,NaN,NaN,NaN,NaN
572255,604807,758868,918865,967471,970952,784715,745441,716579,732103,732103,745441,967471,967471,NaN,NaN,NaN,NaN,NaN,NaN
572256,791040,791040,832786,277358,522132,681203,90781,692730,469116,592612,340530,340530,340530,NaN,NaN,NaN,NaN,NaN,NaN
572257,880639,722323,825005,247342,832762,953126,766052,82260,126631,82260,716463,953126,953126,722323.0,825005.0,247342.0,832762.0,953126.0,1.0


In [77]:
for k in ['previous4_song_index', 'previous3_song_index', 'previous2_song_index', 'previous1_song_index', 'proportion']:
    del gram4_predict[k]
gram4_predict

,16,17,18,19,20,21,22,23,24,25,p,p2,p3,song_index
0,962778,913263,910165,194387,722034,624433,901147,810092,74420,250714,722034,722034,722034,NaN
1,659818,415453,307906,523383,672083,52476,249524,514925,495260,130336,672083,364625,364625,NaN
2,332935,133500,606328,578482,261314,203739,402134,129395,230525,946149,261314,261314,261314,NaN
3,503008,824980,267047,238820,709693,453065,709693,453065,709693,453065,709693,709693,709693,NaN
4,7267,589816,482509,77515,246245,186586,338120,186586,338120,130652,365352,365352,365352,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572254,506580,505242,614405,198412,422547,615766,560313,553343,624094,258244,198412,615766,615766,NaN
572255,604807,758868,918865,967471,970952,784715,745441,716579,732103,732103,745441,967471,967471,NaN
572256,791040,791040,832786,277358,522132,681203,90781,692730,469116,592612,340530,340530,340530,NaN
572257,880639,722323,825005,247342,832762,953126,766052,82260,126631,82260,716463,953126,953126,953126.0


In [82]:
(gram4_predict['song_index'] == gram4_predict[21]).mean()

0.19333728259407018

In [83]:
gram4_predict['p4'] = gram4_predict['song_index'].fillna(gram4_predict['p']).astype(int)
(gram4_predict['p4'] == gram4_predict[21]).mean()

0.21951948331087848

In [84]:
r = gram4_predict[[21,22,23,24,25]] == gram4_predict[['p4']].values
r[25] = r[25] & ~(r[[21,22,23,24]].any(axis=1))
r[24] = r[24] & ~(r[[21,22,23]].any(axis=1))
r[23] = r[23] & ~(r[[21,22]].any(axis=1))
r[22] = r[22] & ~(r[[21]].any(axis=1))
w = [1., 0.63, 0.5, 0.43, 0.38]
all_songs = meta_song.shape[0]
dcg = r.mean() @ w
predict_songs = gram4_predict['p4'].sample(frac=0.25).nunique()
fill_songs = 4*test_source['session_id'].nunique()
coverage = (predict_songs+fill_songs)/all_songs
rich.print({'dcg':dcg, 'coverage':coverage, 'total':dcg*0.8+coverage*0.2})

{'dcg': 0.23705600086674045, 'coverage': 0.6012872655019055, 'total': 0.30990225379377345}

In [85]:
gram4_predict

,16,17,18,19,20,21,22,23,24,25,p,p2,p3,song_index,p4
0,962778,913263,910165,194387,722034,624433,901147,810092,74420,250714,722034,722034,722034,NaN,722034
1,659818,415453,307906,523383,672083,52476,249524,514925,495260,130336,672083,364625,364625,NaN,672083
2,332935,133500,606328,578482,261314,203739,402134,129395,230525,946149,261314,261314,261314,NaN,261314
3,503008,824980,267047,238820,709693,453065,709693,453065,709693,453065,709693,709693,709693,NaN,709693
4,7267,589816,482509,77515,246245,186586,338120,186586,338120,130652,365352,365352,365352,NaN,365352
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572254,506580,505242,614405,198412,422547,615766,560313,553343,624094,258244,198412,615766,615766,NaN,198412
572255,604807,758868,918865,967471,970952,784715,745441,716579,732103,732103,745441,967471,967471,NaN,745441
572256,791040,791040,832786,277358,522132,681203,90781,692730,469116,592612,340530,340530,340530,NaN,340530
572257,880639,722323,825005,247342,832762,953126,766052,82260,126631,82260,716463,953126,953126,953126.0,953126


In [86]:
for k in ['song_index']:
    del gram4_predict[k]
gram4_predict

,16,17,18,19,20,21,22,23,24,25,p,p2,p3,p4
0,962778,913263,910165,194387,722034,624433,901147,810092,74420,250714,722034,722034,722034,722034
1,659818,415453,307906,523383,672083,52476,249524,514925,495260,130336,672083,364625,364625,672083
2,332935,133500,606328,578482,261314,203739,402134,129395,230525,946149,261314,261314,261314,261314
3,503008,824980,267047,238820,709693,453065,709693,453065,709693,453065,709693,709693,709693,709693
4,7267,589816,482509,77515,246245,186586,338120,186586,338120,130652,365352,365352,365352,365352
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572254,506580,505242,614405,198412,422547,615766,560313,553343,624094,258244,198412,615766,615766,198412
572255,604807,758868,918865,967471,970952,784715,745441,716579,732103,732103,745441,967471,967471,745441
572256,791040,791040,832786,277358,522132,681203,90781,692730,469116,592612,340530,340530,340530,340530
572257,880639,722323,825005,247342,832762,953126,766052,82260,126631,82260,716463,953126,953126,953126


In [87]:
gram5_predict = pd.merge(gram4_predict, gram5, left_on=[16,17,18,19,20], right_on=['previous5_song_index', 'previous4_song_index', 'previous3_song_index', 'previous2_song_index', 'previous1_song_index'], how='left')
gram5_predict

,16,17,18,19,20,21,22,23,24,25,...,p2,p3,p4,previous5_song_index,previous4_song_index,previous3_song_index,previous2_song_index,previous1_song_index,song_index,proportion
0,962778,913263,910165,194387,722034,624433,901147,810092,74420,250714,...,722034,722034,722034,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,659818,415453,307906,523383,672083,52476,249524,514925,495260,130336,...,364625,364625,672083,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,332935,133500,606328,578482,261314,203739,402134,129395,230525,946149,...,261314,261314,261314,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,503008,824980,267047,238820,709693,453065,709693,453065,709693,453065,...,709693,709693,709693,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7267,589816,482509,77515,246245,186586,338120,186586,338120,130652,...,365352,365352,365352,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572254,506580,505242,614405,198412,422547,615766,560313,553343,624094,258244,...,615766,615766,198412,NaN,NaN,NaN,NaN,NaN,NaN,NaN
572255,604807,758868,918865,967471,970952,784715,745441,716579,732103,732103,...,967471,967471,745441,NaN,NaN,NaN,NaN,NaN,NaN,NaN
572256,791040,791040,832786,277358,522132,681203,90781,692730,469116,592612,...,340530,340530,340530,NaN,NaN,NaN,NaN,NaN,NaN,NaN
572257,880639,722323,825005,247342,832762,953126,766052,82260,126631,82260,...,953126,953126,953126,880639.0,722323.0,825005.0,247342.0,832762.0,953126.0,1.0


In [97]:
for k in ['previous5_song_index', 'previous4_song_index', 'previous3_song_index', 'previous2_song_index', 'previous1_song_index', 'proportion']:
    del gram5_predict[k]
gram5_predict

,16,17,18,19,20,21,22,23,24,25,p,p2,p3,p4,song_index
0,962778,913263,910165,194387,722034,624433,901147,810092,74420,250714,722034,722034,722034,722034,NaN
1,659818,415453,307906,523383,672083,52476,249524,514925,495260,130336,672083,364625,364625,672083,NaN
2,332935,133500,606328,578482,261314,203739,402134,129395,230525,946149,261314,261314,261314,261314,NaN
3,503008,824980,267047,238820,709693,453065,709693,453065,709693,453065,709693,709693,709693,709693,NaN
4,7267,589816,482509,77515,246245,186586,338120,186586,338120,130652,365352,365352,365352,365352,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572254,506580,505242,614405,198412,422547,615766,560313,553343,624094,258244,198412,615766,615766,198412,NaN
572255,604807,758868,918865,967471,970952,784715,745441,716579,732103,732103,745441,967471,967471,745441,NaN
572256,791040,791040,832786,277358,522132,681203,90781,692730,469116,592612,340530,340530,340530,340530,NaN
572257,880639,722323,825005,247342,832762,953126,766052,82260,126631,82260,716463,953126,953126,953126,953126.0


In [98]:
(gram5_predict['song_index'] == gram5_predict[21]).mean()

0.1816450243683367

In [100]:
gram5_predict['p5'] = gram5_predict['song_index'].fillna(gram5_predict['p']).astype(int)
(gram5_predict['p5'] == gram5_predict[21]).mean()

0.21216442205365052

In [101]:
r = gram5_predict[[21,22,23,24,25]] == gram5_predict[['p5']].values
r[25] = r[25] & ~(r[[21,22,23,24]].any(axis=1))
r[24] = r[24] & ~(r[[21,22,23]].any(axis=1))
r[23] = r[23] & ~(r[[21,22]].any(axis=1))
r[22] = r[22] & ~(r[[21]].any(axis=1))
w = [1., 0.63, 0.5, 0.43, 0.38]
all_songs = meta_song.shape[0]
dcg = r.mean() @ w
predict_songs = gram5_predict['p5'].sample(frac=0.25).nunique()
fill_songs = 4*test_source['session_id'].nunique()
coverage = (predict_songs+fill_songs)/all_songs
rich.print({'dcg':dcg, 'coverage':coverage, 'total':dcg*0.8+coverage*0.2})

{'dcg': 0.229690280100444, 'coverage': 0.6008370912534248, 'total': 0.3039196423310402}

In [104]:
del gram5_predict['song_index']
gram5_predict

,16,17,18,19,20,21,22,23,24,25,p,p2,p3,p4,p5
0,962778,913263,910165,194387,722034,624433,901147,810092,74420,250714,722034,722034,722034,722034,722034
1,659818,415453,307906,523383,672083,52476,249524,514925,495260,130336,672083,364625,364625,672083,672083
2,332935,133500,606328,578482,261314,203739,402134,129395,230525,946149,261314,261314,261314,261314,261314
3,503008,824980,267047,238820,709693,453065,709693,453065,709693,453065,709693,709693,709693,709693,709693
4,7267,589816,482509,77515,246245,186586,338120,186586,338120,130652,365352,365352,365352,365352,365352
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572254,506580,505242,614405,198412,422547,615766,560313,553343,624094,258244,198412,615766,615766,198412,198412
572255,604807,758868,918865,967471,970952,784715,745441,716579,732103,732103,745441,967471,967471,745441,745441
572256,791040,791040,832786,277358,522132,681203,90781,692730,469116,592612,340530,340530,340530,340530,340530
572257,880639,722323,825005,247342,832762,953126,766052,82260,126631,82260,716463,953126,953126,953126,953126


In [111]:
final = gram5_predict.iloc[:,10:]
final

,p,p2,p3,p4,p5
0,722034,722034,722034,722034,722034
1,672083,364625,364625,672083,672083
2,261314,261314,261314,261314,261314
3,709693,709693,709693,709693,709693
4,365352,365352,365352,365352,365352
...,...,...,...,...,...
572254,198412,615766,615766,198412,198412
572255,745441,967471,967471,745441,745441
572256,340530,340530,340530,340530,340530
572257,716463,953126,953126,953126,953126


In [107]:
# song_id_to_index_dict = dict(zip(meta_song['song_id'], meta_song['song_index']))
# # song_id_to_index_dict
index_to_song_id_dict = dict(zip(meta_song['song_index'], meta_song['song_id']))
index_to_song_id_dict

{0: '10a46165bb84e056438e06c35763ee61',
 1: 'ff025522d0f8e7198a75a4e03edce55c',
 2: '7feec55b825a1c93d55b3ef9af9f9be5',
 3: 'ed8b14c6d0fccd1a95f9140933f809f3',
 4: '2ce5e96e0213a6dac83c54b003418726',
 5: 'bc4c16c9a0e60a8b34212f24a8142677',
 6: 'f83417fa016c2d7fbc90c5ccc9068817',
 7: 'cfe5665103b92f32ab424bd1d83f977f',
 8: '9c36969fae42d113e97a9a0f8f255a87',
 9: '55b5febf1ce54c3e08f5ea693fad242f',
 10: '15c91fb2d95cee04b0eb1122b123d848',
 11: '757f039c778531630e9da487ddd5055d',
 12: 'ad3a8117fdc806b261bd861b112e61ff',
 13: '1c1fc1652065a114a44033c7ecc8e2a7',
 14: '92e36ebd1ac3a766120dfa2561f1766b',
 15: '567195f94a0537800759ab2ec66755d9',
 16: 'd25aefbebcfea19f21fe1ab078a893c1',
 17: 'f2a24cef800d7a4da6d2b85275e0511d',
 18: 'e420a5e72b63511e3ccf0923dd88034e',
 19: '61537f8ba5a11b1f17b2b5c390343611',
 20: '4bf1cdb51348bdc3fe6a63c214fb6f9d',
 21: 'b438345789d3af74d149f278cde2c6c2',
 22: 'e8705dac7a2d1e415593b792ed33eaa8',
 23: 'cfd311c55c7c1aa49613313b2a94277c',
 24: '0c229dfa2c30755efb37

In [118]:
final['top1'] = final['p'].map(index_to_song_id_dict)
final['top2'] = final['p2'].map(index_to_song_id_dict)
final['top3'] = final['p3'].map(index_to_song_id_dict)
final['top4'] = final['p4'].map(index_to_song_id_dict)
final['top5'] = final['p5'].map(index_to_song_id_dict)
final

,p,p2,p3,p4,p5,top1,top2,top3,top4,top5
0,722034,722034,722034,722034,722034,824c159701c8553b0e38f0d36ddd6197,824c159701c8553b0e38f0d36ddd6197,824c159701c8553b0e38f0d36ddd6197,824c159701c8553b0e38f0d36ddd6197,824c159701c8553b0e38f0d36ddd6197
1,672083,364625,364625,672083,672083,6309716e08a58c64871c823f44749686,5befdbffa0268f49987f263afd421a6e,5befdbffa0268f49987f263afd421a6e,6309716e08a58c64871c823f44749686,6309716e08a58c64871c823f44749686
2,261314,261314,261314,261314,261314,3b050c1502af19554bb9ef8efc67c00d,3b050c1502af19554bb9ef8efc67c00d,3b050c1502af19554bb9ef8efc67c00d,3b050c1502af19554bb9ef8efc67c00d,3b050c1502af19554bb9ef8efc67c00d
3,709693,709693,709693,709693,709693,e29a8f57507d7e4a926a6bd3d51841d3,e29a8f57507d7e4a926a6bd3d51841d3,e29a8f57507d7e4a926a6bd3d51841d3,e29a8f57507d7e4a926a6bd3d51841d3,e29a8f57507d7e4a926a6bd3d51841d3
4,365352,365352,365352,365352,365352,e8ff873135b65c7ddce395488597f694,e8ff873135b65c7ddce395488597f694,e8ff873135b65c7ddce395488597f694,e8ff873135b65c7ddce395488597f694,e8ff873135b65c7ddce395488597f694
...,...,...,...,...,...,...,...,...,...,...
572254,198412,615766,615766,198412,198412,0f59db4259ec62fcf00399b69786a3c0,3c66c29e5c26efc705c261732b977950,3c66c29e5c26efc705c261732b977950,0f59db4259ec62fcf00399b69786a3c0,0f59db4259ec62fcf00399b69786a3c0
572255,745441,967471,967471,745441,745441,dd45bfa90e9186681877c15eb0166a35,477ecc7fb10644c0d31408030cd464c6,477ecc7fb10644c0d31408030cd464c6,dd45bfa90e9186681877c15eb0166a35,dd45bfa90e9186681877c15eb0166a35
572256,340530,340530,340530,340530,340530,d954ae7d5ccb38784815b6dc3c9904f2,d954ae7d5ccb38784815b6dc3c9904f2,d954ae7d5ccb38784815b6dc3c9904f2,d954ae7d5ccb38784815b6dc3c9904f2,d954ae7d5ccb38784815b6dc3c9904f2
572257,716463,953126,953126,953126,953126,8377a80da218bf35ed0729be7c6efc7a,c31a95cdd5635c6edfbde9143286b9b7,c31a95cdd5635c6edfbde9143286b9b7,c31a95cdd5635c6edfbde9143286b9b7,c31a95cdd5635c6edfbde9143286b9b7


In [119]:
final = final.iloc[:, 5:]

In [120]:
final

,top1,top2,top3,top4,top5
0,824c159701c8553b0e38f0d36ddd6197,824c159701c8553b0e38f0d36ddd6197,824c159701c8553b0e38f0d36ddd6197,824c159701c8553b0e38f0d36ddd6197,824c159701c8553b0e38f0d36ddd6197
1,6309716e08a58c64871c823f44749686,5befdbffa0268f49987f263afd421a6e,5befdbffa0268f49987f263afd421a6e,6309716e08a58c64871c823f44749686,6309716e08a58c64871c823f44749686
2,3b050c1502af19554bb9ef8efc67c00d,3b050c1502af19554bb9ef8efc67c00d,3b050c1502af19554bb9ef8efc67c00d,3b050c1502af19554bb9ef8efc67c00d,3b050c1502af19554bb9ef8efc67c00d
3,e29a8f57507d7e4a926a6bd3d51841d3,e29a8f57507d7e4a926a6bd3d51841d3,e29a8f57507d7e4a926a6bd3d51841d3,e29a8f57507d7e4a926a6bd3d51841d3,e29a8f57507d7e4a926a6bd3d51841d3
4,e8ff873135b65c7ddce395488597f694,e8ff873135b65c7ddce395488597f694,e8ff873135b65c7ddce395488597f694,e8ff873135b65c7ddce395488597f694,e8ff873135b65c7ddce395488597f694
...,...,...,...,...,...
572254,0f59db4259ec62fcf00399b69786a3c0,3c66c29e5c26efc705c261732b977950,3c66c29e5c26efc705c261732b977950,0f59db4259ec62fcf00399b69786a3c0,0f59db4259ec62fcf00399b69786a3c0
572255,dd45bfa90e9186681877c15eb0166a35,477ecc7fb10644c0d31408030cd464c6,477ecc7fb10644c0d31408030cd464c6,dd45bfa90e9186681877c15eb0166a35,dd45bfa90e9186681877c15eb0166a35
572256,d954ae7d5ccb38784815b6dc3c9904f2,d954ae7d5ccb38784815b6dc3c9904f2,d954ae7d5ccb38784815b6dc3c9904f2,d954ae7d5ccb38784815b6dc3c9904f2,d954ae7d5ccb38784815b6dc3c9904f2
572257,8377a80da218bf35ed0729be7c6efc7a,c31a95cdd5635c6edfbde9143286b9b7,c31a95cdd5635c6edfbde9143286b9b7,c31a95cdd5635c6edfbde9143286b9b7,c31a95cdd5635c6edfbde9143286b9b7


In [125]:
final_data=pd.concat([pd.DataFrame(final.index), final], ignore_index=True, axis=1)
final_data.columns = ['session_id', 'top1', 'top2', 'top3', 'top4', 'top5']
final_data

,session_id,top1,top2,top3,top4,top5
0,0,824c159701c8553b0e38f0d36ddd6197,824c159701c8553b0e38f0d36ddd6197,824c159701c8553b0e38f0d36ddd6197,824c159701c8553b0e38f0d36ddd6197,824c159701c8553b0e38f0d36ddd6197
1,1,6309716e08a58c64871c823f44749686,5befdbffa0268f49987f263afd421a6e,5befdbffa0268f49987f263afd421a6e,6309716e08a58c64871c823f44749686,6309716e08a58c64871c823f44749686
2,2,3b050c1502af19554bb9ef8efc67c00d,3b050c1502af19554bb9ef8efc67c00d,3b050c1502af19554bb9ef8efc67c00d,3b050c1502af19554bb9ef8efc67c00d,3b050c1502af19554bb9ef8efc67c00d
3,3,e29a8f57507d7e4a926a6bd3d51841d3,e29a8f57507d7e4a926a6bd3d51841d3,e29a8f57507d7e4a926a6bd3d51841d3,e29a8f57507d7e4a926a6bd3d51841d3,e29a8f57507d7e4a926a6bd3d51841d3
4,4,e8ff873135b65c7ddce395488597f694,e8ff873135b65c7ddce395488597f694,e8ff873135b65c7ddce395488597f694,e8ff873135b65c7ddce395488597f694,e8ff873135b65c7ddce395488597f694
...,...,...,...,...,...,...
572254,572254,0f59db4259ec62fcf00399b69786a3c0,3c66c29e5c26efc705c261732b977950,3c66c29e5c26efc705c261732b977950,0f59db4259ec62fcf00399b69786a3c0,0f59db4259ec62fcf00399b69786a3c0
572255,572255,dd45bfa90e9186681877c15eb0166a35,477ecc7fb10644c0d31408030cd464c6,477ecc7fb10644c0d31408030cd464c6,dd45bfa90e9186681877c15eb0166a35,dd45bfa90e9186681877c15eb0166a35
572256,572256,d954ae7d5ccb38784815b6dc3c9904f2,d954ae7d5ccb38784815b6dc3c9904f2,d954ae7d5ccb38784815b6dc3c9904f2,d954ae7d5ccb38784815b6dc3c9904f2,d954ae7d5ccb38784815b6dc3c9904f2
572257,572257,8377a80da218bf35ed0729be7c6efc7a,c31a95cdd5635c6edfbde9143286b9b7,c31a95cdd5635c6edfbde9143286b9b7,c31a95cdd5635c6edfbde9143286b9b7,c31a95cdd5635c6edfbde9143286b9b7


In [154]:
# sample = pd.DataFrame(test_source['session_id'].unique(), columns=["session_id"])

sample = pd.read_csv('sample.csv', usecols=['session_id', 'top1', 'top2', 'top3', 'top4', 'top5'])
sample = pd.DataFrame(sample['session_id'])

In [155]:
result = sample.merge(final_data[['session_id', 'top1', 'top2', 'top3', 'top4', 'top5']], on='session_id', how='left')
result

,session_id,top1,top2,top3,top4,top5
0,263698,7520bf419f5a42e79659b8c9f6699cfc,7520bf419f5a42e79659b8c9f6699cfc,7520bf419f5a42e79659b8c9f6699cfc,7520bf419f5a42e79659b8c9f6699cfc,7520bf419f5a42e79659b8c9f6699cfc
1,10768,ba2f3539de5a2af73fe030d2bf6a4d1e,ba2f3539de5a2af73fe030d2bf6a4d1e,ba2f3539de5a2af73fe030d2bf6a4d1e,ba2f3539de5a2af73fe030d2bf6a4d1e,ba2f3539de5a2af73fe030d2bf6a4d1e
2,369848,3cc413223e81ac8a19fffea026d33644,861521e7d6ea993b60a7a1cdee4ef943,861521e7d6ea993b60a7a1cdee4ef943,3cc413223e81ac8a19fffea026d33644,3cc413223e81ac8a19fffea026d33644
3,223379,b4419d918abaf2f2b9bfd8a9256d509c,af913db3de124e78559b2267f2a4d129,af913db3de124e78559b2267f2a4d129,b4419d918abaf2f2b9bfd8a9256d509c,b4419d918abaf2f2b9bfd8a9256d509c
4,617939,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
143059,551348,813b6c953333889e2a926afeefa4293a,7e44c7c1eb2b6f3d581e2d20fdfb84fd,7e44c7c1eb2b6f3d581e2d20fdfb84fd,813b6c953333889e2a926afeefa4293a,813b6c953333889e2a926afeefa4293a
143060,599399,NaN,NaN,NaN,NaN,NaN
143061,518478,d496ec58602e887814ca69fb2037a272,d496ec58602e887814ca69fb2037a272,d496ec58602e887814ca69fb2037a272,d496ec58602e887814ca69fb2037a272,d496ec58602e887814ca69fb2037a272
143062,659668,NaN,NaN,NaN,NaN,NaN


In [156]:
result.isna().sum()
# result.mode().iloc[0]
# result['top4'].value_counts()
result = result.apply(lambda col: col.fillna(col.mode().iloc[0]))
result

,session_id,top1,top2,top3,top4,top5
0,263698,7520bf419f5a42e79659b8c9f6699cfc,7520bf419f5a42e79659b8c9f6699cfc,7520bf419f5a42e79659b8c9f6699cfc,7520bf419f5a42e79659b8c9f6699cfc,7520bf419f5a42e79659b8c9f6699cfc
1,10768,ba2f3539de5a2af73fe030d2bf6a4d1e,ba2f3539de5a2af73fe030d2bf6a4d1e,ba2f3539de5a2af73fe030d2bf6a4d1e,ba2f3539de5a2af73fe030d2bf6a4d1e,ba2f3539de5a2af73fe030d2bf6a4d1e
2,369848,3cc413223e81ac8a19fffea026d33644,861521e7d6ea993b60a7a1cdee4ef943,861521e7d6ea993b60a7a1cdee4ef943,3cc413223e81ac8a19fffea026d33644,3cc413223e81ac8a19fffea026d33644
3,223379,b4419d918abaf2f2b9bfd8a9256d509c,af913db3de124e78559b2267f2a4d129,af913db3de124e78559b2267f2a4d129,b4419d918abaf2f2b9bfd8a9256d509c,b4419d918abaf2f2b9bfd8a9256d509c
4,617939,bc58d186329eda8b23e510ba98b2c1ea,c75e75a6baf65a3c26dbe641422d4e72,c75e75a6baf65a3c26dbe641422d4e72,c75e75a6baf65a3c26dbe641422d4e72,c75e75a6baf65a3c26dbe641422d4e72
...,...,...,...,...,...,...
143059,551348,813b6c953333889e2a926afeefa4293a,7e44c7c1eb2b6f3d581e2d20fdfb84fd,7e44c7c1eb2b6f3d581e2d20fdfb84fd,813b6c953333889e2a926afeefa4293a,813b6c953333889e2a926afeefa4293a
143060,599399,bc58d186329eda8b23e510ba98b2c1ea,c75e75a6baf65a3c26dbe641422d4e72,c75e75a6baf65a3c26dbe641422d4e72,c75e75a6baf65a3c26dbe641422d4e72,c75e75a6baf65a3c26dbe641422d4e72
143061,518478,d496ec58602e887814ca69fb2037a272,d496ec58602e887814ca69fb2037a272,d496ec58602e887814ca69fb2037a272,d496ec58602e887814ca69fb2037a272,d496ec58602e887814ca69fb2037a272
143062,659668,bc58d186329eda8b23e510ba98b2c1ea,c75e75a6baf65a3c26dbe641422d4e72,c75e75a6baf65a3c26dbe641422d4e72,c75e75a6baf65a3c26dbe641422d4e72,c75e75a6baf65a3c26dbe641422d4e72


In [157]:

result.to_csv('result.csv',index=0)